In [1]:
import torch
import torch.nn.functional as F
from torch import nn

In [ ]:
# 首先，构造一个没有任何参数的自定义层。
# 下面的CenteredLayer类要从输入中减去均值。构建它，我们只需1. 继承基础层类 2.实现前向传播功能
class CenteredLayer(nn.Module):
    """将数据进行中心化处理，即将输入的均值调整为0"""
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

In [4]:
# 让我们向该层提供一些数据，验证它是否能按预期工作
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [5]:
# 现在，我们将层作为组件合并到更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [ ]:
Y = net(torch.rand(4, 8))  # 生成在[0, 1)区间内均匀分布的随机数，再将这些随机数中心化
Y.mean()  # 健全性检查：检查均值是否为0.但是由于我们处理的是浮点数，因为存储精度的原因，我们仍然可能会看到一个非常小的非灵数

tensor(5.5879e-09, grad_fn=<MeanBackward0>)

In [7]:
# 带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))  # nn.Parameter是PyTorch中的一个特殊类，用于将张量注册为模型的可学习参数
                                                                # 当你将nn.Parameter包装一个张量时，这个张量会自动被加入到模型的参数列表中，
                                                                # 通过调用model.parameters()可以访问，并且在优化过程中被更新
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [8]:
# 实例化MyLinear类并访问其模型参数
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.2620,  0.2415,  0.4855],
        [-0.0794,  0.5416, -0.2760],
        [ 0.3385,  0.3443, -0.4496],
        [-1.4079, -0.6596, -0.3455],
        [-0.1482,  0.7093, -0.1459]], requires_grad=True)

In [9]:
# 使用自定义层直接执行前向传播计算
linear(torch.rand(2, 5))


tensor([[0.1862, 0.0872, 0.0000],
        [0.1413, 0.3737, 0.0000]])

In [10]:
# 我们可以使用自定义层构建模型，就像使用内置的全连接层一样使用自定义层
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[3.9743],
        [2.1407]])